In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from src.utils.datasets.ggimages import OpenImage
from src.utils.datasets.transform import RandomHorizontalFlip, Resize, Compose, XyToCenter
import torchvision.transforms as transforms
from src.utils.display.images import imshow, result_show
from torch.utils.data import DataLoader
from src.utils.datasets.adapter import convert_data
import numpy as np
from src.network.yolo import Yolo
from src.config import VOC_ANCHORS
from src.utils.process_boxes import preprocess_true_boxes
from src.config import IOU_THRESHOLD, TENSORBOARD_PATH
from tensorboardX import SummaryWriter
from datetime import datetime
import time
from torch.optim import SGD, RMSprop, Adam
from torch.optim.lr_scheduler import StepLR
from src.utils.evaluate.metter import AverageMeter




general_transform = Compose([
    Resize((448, 448)),
    XyToCenter()
])


transform = transforms.Compose([
                transforms.ToTensor()
            ])

In [3]:
batch_size = 48

ds = OpenImage('/data/data', 'OpenImage', general_transform=general_transform, transform=transform)
ds_val = OpenImage('/data/data', 'OpenImage_Val', general_transform=general_transform, transform=transform)


train_data_loader = DataLoader(ds, batch_size=batch_size, shuffle=True, collate_fn=convert_data, num_workers=4, drop_last=True)
val_data_loader = DataLoader(ds_val, batch_size=batch_size , shuffle=True, collate_fn=convert_data, num_workers=4, drop_last=True)

print(ds.classes)
print(ds_val.classes)

['__background__', 'Tree', 'Flower']
['__background__', 'Tree', 'Flower']


In [4]:
from src.network.base import DarkNet, DarknetBody, YoloBody

model = Yolo(VOC_ANCHORS, ds.classes)
model.cuda()
model.train()

Yolo(
  (yolo_body): DataParallel(
    (module): YoloBody(
      (body_bottom): DarknetBodyBottom(
        (first_layer): Conv2d(
          (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (relu): LeakyReLU(negative_slope=0.1)
        )
        (second_layer): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (third_layer): Conv2d(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (relu): LeakyReLU(negative_slope=0.1)
        )
        (forth_layer): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (fifth_layer): BottleneckBlock(
          (first_layer): Conv2d(
            (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1),

In [5]:
params = filter(lambda x: x.requires_grad, model.parameters())
# optimizer = SGD(model.parameters(), lr = 0.0001, momentum=0.9)
optimizer = RMSprop(model.parameters(), lr=1e-5, momentum=0.9, weight_decay=0.00005)
exp_lr_scheduler = StepLR(optimizer, step_size=100, gamma=0.9)

In [6]:
def val(data_val_gen ,model):
    val_loss = AverageMeter()
    with torch.no_grad():
        model.eval()
        for step, blobs in enumerate(data_val_gen):
            batch_tensor, batch_boxes, detectors_mask, matching_true_boxes, im_info, img_name = blobs
            batch_tensor = batch_tensor.to(torch.device('cuda'))
            detectors_mask = detectors_mask.to(torch.device('cuda'))
            matching_true_boxes = matching_true_boxes.to(torch.device('cuda'))
            batch_boxes = batch_boxes.to(torch.device('cuda'))
            output = model(batch_tensor)
            loss = model.loss(output, batch_boxes, detectors_mask, matching_true_boxes)
            val_loss.update(loss.item())
    return val_loss

In [7]:
def train(data_gen, data_val_gen ,model, metters, optimizer, lr_scheduler, tensorboard_writer, current_epoch=0):
    
    steps_per_epoch = len(data_gen) 
    model.train()
    train_loss = metters
    start_time = time.time()

    for step, blobs in enumerate(data_gen):
        batch_tensor, batch_boxes, detectors_mask, matching_true_boxes, im_info, img_name = blobs
        batch_tensor = batch_tensor.to(torch.device('cuda'))
        detectors_mask = detectors_mask.to(torch.device('cuda'))
        matching_true_boxes = matching_true_boxes.to(torch.device('cuda'))
        batch_boxes = batch_boxes.to(torch.device('cuda'))
        output = model(batch_tensor)
        loss = model.loss(output, batch_boxes, detectors_mask, matching_true_boxes)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        train_loss.update(loss.item())
        del batch_tensor

        current_step = current_epoch * steps_per_epoch + step
        if step % 100 == 1:
            print("epochs time %s" % (time.time() - start_time))
            start_time = time.time()
            tensorboard_writer.add_scalar("loss", train_loss.avg, (current_epoch * steps_per_epoch) + step)
            log_text = 'epoch: %d : step %d,  loss: %.4f at %s' % (
                current_epoch + 1, step , train_loss.avg, datetime.now().strftime('%m/%d_%H:%M'))
            print(log_text)
            

#         if step % 200 == 1:
#             print("Validate")
#             val_loss = val(data_val_gen, model)
#             log_text = 'epoch: %d : step %d,  val_loss: %.4f at %s' % (
#                 current_epoch + 1, step , val_loss.avg, datetime.now().strftime('%m/%d_%H:%M'))
#             print(log_text)
#             tensorboard_writer.add_scalar("val_loss", val_loss.avg, (current_epoch * steps_per_epoch) + step)


In [8]:
writer = SummaryWriter("%s/%s" % (TENSORBOARD_PATH , datetime.now().strftime('%m/%d_%H:%M')))
train_loss = AverageMeter()
for i in range(6):
    train(train_data_loader, val_data_loader ,model, train_loss, optimizer, exp_lr_scheduler, writer,i)
    torch.save(model.state_dict(), './save_model/model_without_val_%s.pth' % i)

/opt/conda/envs/pytorch/lib/python3.7/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


epochs time 13.092717170715332
epoch: 1 : step 1,  loss: 6.0463 at 04/15_05:26
epochs time 51.70197415351868
epoch: 1 : step 101,  loss: 1.8780 at 04/15_05:27
epochs time 51.82934308052063
epoch: 1 : step 201,  loss: 1.3210 at 04/15_05:28
epochs time 51.971285820007324
epoch: 1 : step 301,  loss: 1.1202 at 04/15_05:29
epochs time 52.20403170585632
epoch: 1 : step 401,  loss: 1.0202 at 04/15_05:30
epochs time 52.35761570930481
epoch: 1 : step 501,  loss: 0.9425 at 04/15_05:31
epochs time 52.45239233970642
epoch: 1 : step 601,  loss: 0.8825 at 04/15_05:31
epochs time 52.97049140930176
epoch: 1 : step 701,  loss: 0.8459 at 04/15_05:32
epochs time 54.04622507095337
epoch: 1 : step 801,  loss: 0.8167 at 04/15_05:33
epochs time 54.46024179458618
epoch: 1 : step 901,  loss: 0.7976 at 04/15_05:34
epochs time 54.63852405548096
epoch: 1 : step 1001,  loss: 0.7793 at 04/15_05:35
epochs time 55.53338122367859
epoch: 1 : step 1101,  loss: 0.7618 at 04/15_05:36
epochs time 54.944992780685425
epoch: 

KeyboardInterrupt: 

In [9]:
torch.save(model.state_dict(), './model.pth')

In [ ]:
from src.utils.display.images import imshow, result_show
for k in range(batch_tensor.shape[0]):
    current_im_info = im_info[k]
    tmp = batch_boxes[k] * torch.Tensor([current_im_info[0], current_im_info[1], current_im_info[0], current_im_info[1], 1])
    tmp = tmp.numpy()        
    between = tmp[:, 2:4] / 2        
    xy = tmp[:, :2]
    xy_min = xy - between
    xy_max = xy + between
    print(np.hstack((xy_min, xy_max)))
    imshow(batch_tensor[k], gt_boxes=np.hstack((xy_min, xy_max)))
    break

In [ ]:
model.train()
batch_tensor = batch_tensor.to(torch.device('cuda'))
detectors_mask = detectors_mask.to(torch.device('cuda'))
matching_true_boxes = matching_true_boxes.to(torch.device('cuda'))
batch_boxes = batch_boxes.to(torch.device('cuda'))

output = model(batch_tensor)
loss = model.loss(output, batch_boxes, detectors_mask, matching_true_boxes)
print(loss.item())

In [ ]:
model.train()
boxes, scores, classes = model.predict(batch_tensor)

In [ ]:
print(boxes, scores, classes)

In [ ]:
for k in range(batch_tensor.shape[0]):
    current_im_info = im_info[k]
    tmp = boxes.cpu()
    tmp = tmp.detach().numpy()       
    imshow(batch_tensor[k].cpu(), gt_boxes=tmp)
    break